In [1]:
import os
import cv2
import mmcv
import os.path as osp
import decord
import copy as cp
import numpy as np
# import matplotlib.pyplot as plt
import urllib
import moviepy.editor as mpy
# import random as rd

import warnings


import mmengine
import math

# from mmpose.apis import vis_pose_result
# from mmpose.models import TopDown
# from mmpose.models.pose_estimators import TopdownPoseEstimator

from mmengine import load #, dump

from mmaction.datasets import (CenterCrop,
                               GeneratePoseTarget,
                               PoseCompact, PoseDecode,
                               Resize)

# We assume the annotation is already prepared
gym_ann_file = '../data/gym/gym_hrnet.pkl'  # https://download.openmmlab.com/mmaction/pyskl/data/gym/gym_hrnet.pkl
ntu60_ann_file = '../data/nturgbd/ntu60_hrnet.pkl'  # https://download.openmmlab.com/mmaction/pyskl/data/nturgbd/ntu60_hrnet.pkl

In [2]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.6
FONTCOLOR = (255, 255, 255)
BGBLUE = (0, 119, 182)
THICKNESS = 1
LINETYPE = 1

In [3]:
def vis_pose_result(model,
                    img,
                    result,
                    radius=4,
                    thickness=1,
                    kpt_score_thr=0.3,
                    bbox_color='green',
                    dataset='TopDownCocoDataset',
                    dataset_info=None,
                    show=False,
                    out_file=None):
    """Visualize the detection results on the image.

    Args:
        model (nn.Module): The loaded detector.
        img (str | np.ndarray): Image filename or loaded image.
        result (list[dict]): The results to draw over `img`
                (bbox_result, pose_result).
        radius (int): Radius of circles.
        thickness (int): Thickness of lines.
        kpt_score_thr (float): The threshold to visualize the keypoints.
        skeleton (list[tuple()]): Default None.
        show (bool):  Whether to show the image. Default True.
        out_file (str|None): The filename of the output visualization image.
    """

    warnings.warn(
        'dataset is deprecated.'
        'Please set `dataset_info` in the config.'
        'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
        DeprecationWarning)
    # TODO: These will be removed in the later versions.
    palette = np.array([[255, 128, 0], [255, 153, 51], [255, 178, 102],
                        [230, 230, 0], [255, 153, 255], [153, 204, 255],
                        [255, 102, 255], [255, 51, 255], [102, 178, 255],
                        [51, 153, 255], [255, 153, 153], [255, 102, 102],
                        [255, 51, 51], [153, 255, 153], [102, 255, 102],
                        [51, 255, 51], [0, 255, 0], [0, 0, 255],
                        [255, 0, 0], [255, 255, 255]])

    if dataset in ('TopDownCocoDataset', 'BottomUpCocoDataset',
                   'TopDownOCHumanDataset', 'AnimalMacaqueDataset'):
        # show the results
        # skeleton = [[15, 13], [13, 11], [16, 14], [14, 12], [11, 12],
        #             [5, 11], [6, 12], [5, 6], [5, 7], [6, 8], [7, 9],
        #             [8, 10], [1, 2], [0, 1], [0, 2], [1, 3], [2, 4],
        #             [3, 5], [4, 6]]
        # skeleton = [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13],
        #             [6, 12], [7, 13], [6, 7], [6, 8], [7, 9], [8, 10],
        #             [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5],
        #             [4, 6], [5, 7]]
        # pose_link_color = palette[[
        #     0, 0, 0, 0, 7, 7, 7, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16
        # ]]
        # pose_kpt_color = palette[[
        #     16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0
        # ]]
        
        skeleton = [[16, 14], [14, 12], [6, 8], [8, 10],
                    [12, 13], [6, 12], [7, 13], [6, 7],
                    [0, 15], [15, 13], [7, 9], [9, 11],
                    [2, 3], [1, 2], [1, 3],
                    [2, 4], [4, 6], [3, 5], [5, 7]
                   ]
        pose_link_color = palette[[
            16, 16, 16, 16, 9, 9, 9, 9, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9, 9
        ]]
        pose_kpt_color = palette[[
            9, 9, 9, 9, 9, 16, 0, 16, 0, 16, 0, 16, 0, 16, 0, 16, 0
        ]]

    else:
        NotImplementedError()

    img = show_result(
        img,
        result,
        skeleton,
        kpt_score_thr=kpt_score_thr,
        bbox_color=bbox_color,
        pose_kpt_color=pose_kpt_color,
        pose_limb_color=pose_link_color,
        radius=radius,
        thickness=thickness,
        show=show,
        out_file=out_file)

    return img


def show_result(img,
                result,
                skeleton=None,
                kpt_score_thr=0.3,
                bbox_color='green',
                pose_kpt_color=None,
                pose_limb_color=None,
                radius=4,
                text_color=(255, 0, 0),
                thickness=1,
                font_scale=0.5,
                win_name='',
                show=False,
                wait_time=0,
                out_file=None):
    """Draw `result` over `img`.

    Args:
        img (str or Tensor): The image to be displayed.
        result (list[dict]): The results to draw over `img`
            (bbox_result, pose_result).
        kpt_score_thr (float, optional): Minimum score of keypoints
            to be shown. Default: 0.3.
        bbox_color (str or tuple or :obj:`Color`): Color of bbox lines.
        pose_kpt_color (np.array[Nx3]`): Color of N keypoints.
            If None, do not draw keypoints.
        pose_limb_color (np.array[Mx3]): Color of M limbs.
            If None, do not draw limbs.
        text_color (str or tuple or :obj:`Color`): Color of texts.
        thickness (int): Thickness of lines.
        font_scale (float): Font scales of texts.
        win_name (str): The window name.
        wait_time (int): Value of waitKey param.
            Default: 0.
        out_file (str or None): The filename to write the image.
            Default: None.

    Returns:
        Tensor: Visualized img, only if not `show` or `out_file`.
    """

    img = mmcv.imread(img)
    img = img.copy()
    img_h, img_w, _ = img.shape

    bbox_result = []
    pose_result = []
    for res in result:
        bbox_result.append(res['bbox'])
        pose_result.append(res['keypoints'])

    if len(bbox_result) > 0:
        bboxes = np.vstack(bbox_result)
        # draw bounding boxes
        mmcv.imshow_bboxes(
            img,
            bboxes,
            colors=bbox_color,
            top_k=-1,
            thickness=thickness,
            show=False,
            win_name=win_name,
            wait_time=wait_time,
            out_file=None)

        for person_id, kpts in enumerate(pose_result):
            # draw each point on image
            if pose_kpt_color is not None:
                assert len(pose_kpt_color) == len(kpts), (
                    len(pose_kpt_color), len(kpts))
                for kid, kpt in enumerate(kpts):
                    x_coord, y_coord, kpt_score = int(kpt[0]), int(
                        kpt[1]), kpt[2]
                    if kpt_score > kpt_score_thr:
                        img_copy = img.copy()
                        r, g, b = pose_kpt_color[kid]
                        cv2.circle(img_copy, (int(x_coord), int(y_coord)),
                                   radius, (int(r), int(g), int(b)), -1)
                        transparency = max(0, min(1, kpt_score))
                        cv2.addWeighted(
                            img_copy,
                            transparency,
                            img,
                            1 - transparency,
                            0,
                            dst=img)

            # draw limbs
            if skeleton is not None and pose_limb_color is not None:
                assert len(pose_limb_color) == len(skeleton)
                for sk_id, sk in enumerate(skeleton):
                    pos1 = (int(kpts[sk[0] - 1, 0]), int(kpts[sk[0] - 1,
                                                              1]))
                    pos2 = (int(kpts[sk[1] - 1, 0]), int(kpts[sk[1] - 1,
                                                              1]))
                    if (0 < pos1[0] < img_w and 0 < pos1[1] < img_h
                            and 0 < pos2[0] < img_w and 0 < pos2[1] < img_h
                            and kpts[sk[0] - 1, 2] > kpt_score_thr
                            and kpts[sk[1] - 1, 2] > kpt_score_thr):
                        img_copy = img.copy()
                        X = (pos1[0], pos2[0])
                        Y = (pos1[1], pos2[1])
                        mX = np.mean(X)
                        mY = np.mean(Y)
                        length = ((Y[0] - Y[1])**2 + (X[0] - X[1])**2)**0.5
                        angle = math.degrees(
                            math.atan2(Y[0] - Y[1], X[0] - X[1]))
                        stickwidth = 2
                        polygon = cv2.ellipse2Poly(
                            (int(mX), int(mY)),
                            (int(length / 2), int(stickwidth)), int(angle),
                            0, 360, 1)

                        r, g, b = pose_limb_color[sk_id]
                        cv2.fillConvexPoly(img_copy, polygon,
                                           (int(r), int(g), int(b)))
                        transparency = max(
                            0,
                            min(
                                1, 0.5 *
                                (kpts[sk[0] - 1, 2] + kpts[sk[1] - 1, 2])))
                        cv2.addWeighted(
                            img_copy,
                            transparency,
                            img,
                            1 - transparency,
                            0,
                            dst=img)

    show, wait_time = show, 1
    if show:
        height, width = img.shape[:2]
        max_ = max(height, width)

        factor = min(1, 800 / max_)
        enlarge = cv2.resize(
            img, (0, 0),
            fx=factor,
            fy=factor,
            interpolation=cv2.INTER_CUBIC)
        mmcv.imshow(enlarge, win_name, wait_time)

    if out_file is not None:
        mmcv.imwrite(img, out_file)

    return img

In [4]:
def add_label(frame, label, BGCOLOR=BGBLUE):
    threshold = 30
    def split_label(label):
        label = label.split()
        lines, cline = [], ''
        for word in label:
            if len(cline) + len(word) < threshold:
                cline = cline + ' ' + word
            else:
                lines.append(cline)
                cline = word
        if cline != '':
            lines += [cline]
        return lines
    
    if len(label) > 30:
        label = split_label(label)
    else:
        label = [label]
    label = ['Action: '] + label
    
    sizes = []
    for line in label:
        sizes.append(cv2.getTextSize(line, FONTFACE, FONTSCALE, THICKNESS)[0])
    box_width = max([x[0] for x in sizes]) + 10
    text_height = sizes[0][1]
    box_height = len(sizes) * (text_height + 6)
    
    cv2.rectangle(frame, (0, 0), (box_width, box_height), BGCOLOR, -1)
    for i, line in enumerate(label):
        location = (5, (text_height + 6) * i + text_height + 3)
        cv2.putText(frame, line, location, FONTFACE, FONTSCALE, FONTCOLOR, THICKNESS, LINETYPE)
    return frame
    

def vis_skeleton(vid_path, anno, category_name=None, ratio=0.5):
    vid = decord.VideoReader(vid_path)
    frames = [x.asnumpy() for x in vid]
    
    h, w, _ = frames[0].shape
    new_shape = (int(w * ratio), int(h * ratio))
    frames = [cv2.resize(f, new_shape) for f in frames]
    
    assert len(frames) == anno['total_frames']
    # The shape is N x T x K x 3
    kps = np.concatenate([anno['keypoint'], anno['keypoint_score'][..., None]], axis=-1)
    kps[..., :2] *= ratio
    # Convert to T x N x K x 3
    kps = kps.transpose([1, 0, 2, 3])
    vis_frames = []

    # we need an instance of TopDown model, so build a minimal one
    # model = TopDown(backbone=dict(type='ShuffleNetV1'))
    model = dict(
        type='TopdownPoseEstimator',
        _scope_='mmpose',
        backbone=dict(type='HRNet')
    )
    
    for f, kp in zip(frames, kps):
        bbox = np.zeros([0, 4], dtype=np.float32)
        result = [dict(bbox=bbox, keypoints=k) for k in kp]
        vis_frame = vis_pose_result(model, f, result)

        if category_name is not None:
            vis_frame = add_label(vis_frame, category_name)
        
        vis_frames.append(vis_frame)
    return vis_frames

In [5]:
# left_kp = [1, 3, 5, 7, 9, 11, 13, 15]
# right_kp = [2, 4, 6, 8, 10, 12, 14, 16]

keypoint_pipeline = [
    dict(type=PoseDecode),
    dict(type=PoseCompact, hw_ratio=1., allow_imgpad=True),
    dict(type=Resize, scale=(-1, 64)),
    dict(type=CenterCrop, crop_size=64),
    dict(type=GeneratePoseTarget, sigma=0.6, use_score=True, with_kp=True, with_limb=False
         # , left_kp=left_kp, right_kp=right_kp
        )
]

limb_pipeline = [
    dict(type=PoseDecode),
    dict(type=PoseCompact, hw_ratio=1., allow_imgpad=True),
    dict(type=Resize, scale=(-1, 64)),
    dict(type=CenterCrop, crop_size=64),
    dict(type=GeneratePoseTarget, sigma=0.6, use_score=True, with_kp=False, with_limb=True
         # , left_kp=left_kp, right_kp=right_kp
        )
]

from mmengine.dataset import Compose
def get_pseudo_heatmap(anno, flag='keypoint'):  # maybe related to class GeneratePoseTarget: def generate_a_heatmap()
    assert flag in ['keypoint', 'limb']
    pipeline = Compose(keypoint_pipeline if flag == 'keypoint' else limb_pipeline)
    return pipeline(anno)['imgs']

def vis_heatmaps(heatmaps, channel=-1, ratio=8):
    import matplotlib.cm as cm
    
    if ( 0 <= channel <= heatmaps.shape[1]-1 ):
        heatmaps = [heatmaps[x][channel] for x in range(heatmaps.shape[0])]
    else:  # draw all keypoints / limbs on the same map
        heatmaps = [np.max(x, axis=0) for x in heatmaps]
    
    h, w = heatmaps[0].shape    
    newh, neww = int(h * ratio), int(w * ratio)
    
    cmap = cm.viridis
    heatmaps = [(cmap(x)[..., :3] * 255).astype(np.uint8) for x in heatmaps]

    heatmaps = [cv2.resize(x, (neww, newh)) for x in heatmaps]
    return heatmaps

## GYM

In [ ]:
# Load GYM annotations
lines = list(urllib.request.urlopen('https://sdolivia.github.io/FineGym/resources/dataset/gym99_categories.txt'))
gym_categories = [x.decode().strip().split('; ')[-1] for x in lines]
gym_annos = load(gym_ann_file)['annotations']

In [ ]:
# download sample videos of GYM
!wget https://download.openmmlab.com/mmaction/posec3d/gym_samples.tar
!tar -xf gym_samples.tar
!rm gym_samples.tar

In [ ]:
gym_root = 'gym_samples/'
gym_vids = os.listdir(gym_root)
# visualize pose of which video? index in 0 - 50.
idx = 1
vid = gym_vids[idx]

frame_dir = vid.split('.')[0]
vid_path = osp.join(gym_root, vid)
anno = [x for x in gym_annos if x['frame_dir'] == frame_dir][0]

In [ ]:
vid_path

In [ ]:
# Visualize Skeleton
# vis_frames = vis_skeleton(vid_path, anno, gym_categories[anno['label']])
# vid = mpy.ImageSequenceClip(vis_frames, fps=24)
# vid.ipython_display()

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap)
keypoint_mapvis = [add_label(f, gym_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=24)
vid.ipython_display()

In [ ]:
limb_heatmap = get_pseudo_heatmap(anno, 'keypoint')
limb_mapvis = vis_heatmaps(limb_heatmap)
limb_mapvis = [add_label(f, gym_categories[anno['label']]) for f in limb_mapvis]
vid = mpy.ImageSequenceClip(limb_mapvis, fps=24)
vid.ipython_display()

## NTU60

In [ ]:
# The name list of 
ntu_categories = ['drink water', 'eat meal/snack', 'brushing teeth', 'brushing hair', 'drop', 'pickup', 
                  'throw', 'sitting down', 'standing up (from sitting position)', 'clapping', 'reading', 
                  'writing', 'tear up paper', 'wear jacket', 'take off jacket', 'wear a shoe', 
                  'take off a shoe', 'wear on glasses', 'take off glasses', 'put on a hat/cap', 
                  'take off a hat/cap', 'cheer up', 'hand waving', 'kicking something', 
                  'reach into pocket', 'hopping (one foot jumping)', 'jump up', 
                  'make a phone call/answer phone', 'playing with phone/tablet', 'typing on a keyboard', 
                  'pointing to something with finger', 'taking a selfie', 'check time (from watch)', 
                  'rub two hands together', 'nod head/bow', 'shake head', 'wipe face', 'salute', 
                  'put the palms together', 'cross hands in front (say stop)', 'sneeze/cough', 
                  'staggering', 'falling', 'touch head (headache)', 'touch chest (stomachache/heart pain)', 
                  'touch back (backache)', 'touch neck (neckache)', 'nausea or vomiting condition', 
                  'use a fan (with hand or paper)/feeling warm', 'punching/slapping other person', 
                  'kicking other person', 'pushing other person', 'pat on back of other person', 
                  'point finger at the other person', 'hugging other person', 
                  'giving something to other person', "touch other person's pocket", 'handshaking', 
                  'walking towards each other', 'walking apart from each other']
ntu_annos = load(ntu60_ann_file)

In [ ]:
ntu_annos

In [ ]:
# download sample videos of NTU-60
# !wget https://download.openmmlab.com/mmaction/posec3d/ntu_samples.tar
!tar -xf ntu_samples.tar
!rm ntu_samples.tar

In [ ]:
ntu_root = 'ntu_samples/'
ntu_vids = os.listdir(ntu_root)
# visualize pose of which video? index in 0 - 50.
idx = 20
vid = ntu_vids[idx]

frame_dir = vid.split('.')[0]
vid_path = osp.join(ntu_root, vid)
anno = [x for x in ntu_annos if x['frame_dir'] == frame_dir.split('_')[0]][0]

In [ ]:
anno

In [ ]:
vid_path

In [ ]:
# vis_frames = vis_skeleton(vid_path, anno, ntu_categories[anno['label']])
# vid = mpy.ImageSequenceClip(vis_frames, fps=24)
# vid.ipython_display()

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap)
keypoint_mapvis = [add_label(f, gym_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=24)
vid.ipython_display()

In [ ]:
limb_heatmap = get_pseudo_heatmap(anno, 'keypoint')
limb_mapvis = vis_heatmaps(limb_heatmap)
limb_mapvis = [add_label(f, gym_categories[anno['label']]) for f in limb_mapvis]
vid = mpy.ImageSequenceClip(limb_mapvis, fps=24)
vid.ipython_display()

# NTU60_2D_Single_Person

In [ ]:
ntu60_2d_ann_file = 'mmaction2/tools/data/skeleton/S001C001P001R001A001_rgb.pkl'

In [ ]:
ntu_categories = ['drink water', 'eat meal/snack', 'brushing teeth', 'brushing hair', 'drop', 'pickup', 
                  'throw', 'sitting down', 'standing up (from sitting position)', 'clapping', 'reading', 
                  'writing', 'tear up paper', 'wear jacket', 'take off jacket', 'wear a shoe', 
                  'take off a shoe', 'wear on glasses', 'take off glasses', 'put on a hat/cap', 
                  'take off a hat/cap', 'cheer up', 'hand waving', 'kicking something', 
                  'reach into pocket', 'hopping (one foot jumping)', 'jump up', 
                  'make a phone call/answer phone', 'playing with phone/tablet', 'typing on a keyboard', 
                  'pointing to something with finger', 'taking a selfie', 'check time (from watch)', 
                  'rub two hands together', 'nod head/bow', 'shake head', 'wipe face', 'salute', 
                  'put the palms together', 'cross hands in front (say stop)', 'sneeze/cough', 
                  'staggering', 'falling', 'touch head (headache)', 'touch chest (stomachache/heart pain)', 
                  'touch back (backache)', 'touch neck (neckache)', 'nausea or vomiting condition', 
                  'use a fan (with hand or paper)/feeling warm', 'punching/slapping other person', 
                  'kicking other person', 'pushing other person', 'pat on back of other person', 
                  'point finger at the other person', 'hugging other person', 
                  'giving something to other person', "touch other person's pocket", 'handshaking', 
                  'walking towards each other', 'walking apart from each other']
# ntu2d_annos = load(ntu60_2d_ann_file)['annotations']

In [ ]:
ntu2d_root = 'mmaction2/tools/data/skeleton/'
vid = 'S001C001P001R001A001_rgb.avi'

# frame_dir = vid.split('.')[0]
vid_path = osp.join(ntu2d_root, vid)
# anno = [x for x in ntu2d_annos if x['frame_dir'] == frame_dir.split('_')[0]][0]
anno = load(ntu60_2d_ann_file)

In [ ]:
# anno

In [ ]:
# Visualize Skeleton
vis_frames = vis_skeleton(vid_path, anno, ntu_categories[anno['label']])
vid = mpy.ImageSequenceClip(vis_frames, fps=24)
vid.ipython_display()

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap, channel=-1)
keypoint_mapvis = [add_label(f, ntu_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=24)

# CHECK ndarray.shape, RESTART KERNEL if ERROR!
print(keypoint_heatmap.shape)  # frames X kpts X width X heighs

vid.write_videofile("S001C001P001R001A001_rgb_heatmap.mp4", remove_temp=True)
vid.ipython_display()

In [ ]:
limb_heatmap = get_pseudo_heatmap(anno, 'limb')
limb_mapvis = vis_heatmaps(limb_heatmap)
limb_mapvis = [add_label(f, ntu_categories[anno['label']]) for f in limb_mapvis]
vid = mpy.ImageSequenceClip(limb_mapvis, fps=24)

# CHECK ndarray.shape, RESTART KERNEL if ERROR!
print(limb_heatmap.shape[0])  # frames X kpts X width X heighs

vid.ipython_display()

# NTU60_2D_Multi_Persons

In [ ]:
# ntu60_2d_ann_file = 'mmaction2/tools/data/skeleton/S013C002P018R001A060_rgb.pkl'
ntu60_2d_ann_file = 'data/anno/DJI_0011_12r_10s_1.pkl'

In [ ]:
ntu_categories = ['drink water', 'eat meal/snack', 'brushing teeth', 'brushing hair', 'drop', 'pickup', 
                  'throw', 'sitting down', 'standing up (from sitting position)', 'clapping', 'reading', 
                  'writing', 'tear up paper', 'wear jacket', 'take off jacket', 'wear a shoe', 
                  'take off a shoe', 'wear on glasses', 'take off glasses', 'put on a hat/cap', 
                  'take off a hat/cap', 'cheer up', 'hand waving', 'kicking something', 
                  'reach into pocket', 'hopping (one foot jumping)', 'jump up', 
                  'make a phone call/answer phone', 'playing with phone/tablet', 'typing on a keyboard', 
                  'pointing to something with finger', 'taking a selfie', 'check time (from watch)', 
                  'rub two hands together', 'nod head/bow', 'shake head', 'wipe face', 'salute', 
                  'put the palms together', 'cross hands in front (say stop)', 'sneeze/cough', 
                  'staggering', 'falling', 'touch head (headache)', 'touch chest (stomachache/heart pain)', 
                  'touch back (backache)', 'touch neck (neckache)', 'nausea or vomiting condition', 
                  'use a fan (with hand or paper)/feeling warm', 'punching/slapping other person', 
                  'kicking other person', 'pushing other person', 'pat on back of other person', 
                  'point finger at the other person', 'hugging other person', 
                  'giving something to other person', "touch other person's pocket", 'handshaking', 
                  'walking towards each other', 'walking apart from each other']
# ntu2d_annos = load(ntu60_2d_ann_file)['annotations']

In [ ]:
ntu2d_root = '../cut'
vid = 'DJI_0011_12r_10s_1.mp4'
out_filename = 'data/DJI_0011_12r_10s_1_heatmap_17.MP4'

# frame_dir = vid.split('.')[0]
vid_path = osp.join(ntu2d_root, vid)
# anno = [x for x in ntu2d_annos if x['frame_dir'] == frame_dir.split('_')[0]][0]
anno = load(ntu60_2d_ann_file)

In [ ]:
vid_path

In [ ]:
# with open('S013C002P018R001A060_rgb.txt','w') as data:  
#       data.write(str(anno))

In [ ]:
# Visualize Skeleton
# vis_frames = vis_skeleton(vid_path, anno[0], ntu_categories[anno[0]['label']])
# vid = mpy.ImageSequenceClip(vis_frames, fps=24)
# vid.ipython_display()

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno[0])
keypoint_mapvis = vis_heatmaps(keypoint_heatmap, channel=16)
# keypoint_mapvis = [add_label(f, ntu_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=12)

# CHECK ndarray.shape, RESTART KERNEL if ERROR!
print(keypoint_heatmap.shape)  # frames X kpts X width X heighs

vid.write_videofile(out_filename, remove_temp=True)
# vid.ipython_display()

# CIIS

In [ ]:
ciis_ann_file = 'data/skeleton/ciis_0s7_v1.pkl'

In [ ]:
ciis_categories = ['berdiri', 'berjalan', 'berjongkok', 'merayap', 'melempar', 'membidik (l. panjang)', 
                  'membidik (l. pendek)', 'memukul', 'menendang', 'menusuk']
ciis_annos = load(ciis_ann_file)

In [ ]:
len(ciis_annos['split']['xsub_train'])

In [ ]:
len(ciis_annos['split']['xsub_val'])

In [ ]:
count = np.zeros(len(ciis_categories))
for anno in ciis_annos['annotations']:
    count[anno['label']] += 1
for i, category in enumerate(ciis_categories):
    print(category + ' = ' + str(int(count[i])))

In [ ]:
ciis_annos['split']['xsub_train'][3000:4000]

In [ ]:
ciis_annos['split']['xsub_val'][0:1000]

In [ ]:
ciis_annos['annotations'][:1000]

In [ ]:
ciis_annos['annotations'][2000:3000]

In [ ]:
ciis_annos['annotations'][3000:4000]

In [ ]:
ciis_annos['annotations'][4000:5000]

In [ ]:
ciis_annos['annotations'][5000:6000]

In [ ]:
ciis_annos['annotations'][6000:]

In [ ]:
ciis_root = 'data/video'
ciis_vids = os.listdir(ciis_root)
# visualize pose of which video? index in 0 - 50.
idx = 0
vid = ciis_vids[idx]

frame_dir = '30d_1s1_806.002_282'  # video_id frame_id.person_id data_id
vid_path = osp.join(ciis_root, vid)
anno = [x for x in ciis_annos['annotations'] if x['frame_dir'] == frame_dir][0]

In [ ]:
anno

In [ ]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap)
keypoint_mapvis = [add_label(f, ciis_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=24)
vid.ipython_display()

# Experimental

In [21]:
_ann_file = 'data/train/50d_2s2.pkl'

In [22]:
_annos = load(_ann_file)

In [23]:
type(_annos)

list

In [24]:
_annos

[{'frame_dir': '50d_2s2_34.001_0',
  'label': 7,
  'img_shape': (720, 1280),
  'original_shape': (720, 1280),
  'num_clips': 1,
  'total_frames': 4,
  'clip_len': 4,
  'keypoint': array([[[[600.67248535, 193.74690247],
           [600.67248535, 183.92063904],
           [595.75933838, 188.83377075],
           [541.71490479, 183.92063904],
           [585.93310547, 193.74690247],
           [517.14923096, 203.57316589],
           [571.19372559, 272.35702515],
           [487.67044067, 139.70243835],
           [576.10681152, 350.96713257],
           [536.80175781, 100.39738464],
           [585.93310547, 316.57519531],
           [507.32296753, 355.88024902],
           [546.62805176, 370.61965942],
           [536.80175781, 400.09844971],
           [600.67248535, 444.31665039],
           [536.80175781, 454.14291382],
           [605.58563232, 527.83990479]],
  
          [[602.34777832, 190.6166687 ],
           [598.15643311, 186.42532349],
           [598.15643311, 186.42532349]